In [ ]:
import math
import timeit
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from Crypto.Hash import SHA3_256, SHA256, HMAC
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import random
import re
import json
API_URL = 'http://cryptlygos.pythonanywhere.com'

def sign(k, m, decode = True):
    global P
    R = k * P
    r = R.x % n
    
    if decode:
        h = int.from_bytes(SHA3_256.new(m.to_bytes((m.bit_length() + 7) // 8, byteorder= "big") + r.to_bytes((r.bit_length() + 7) // 8, byteorder="big")).digest(), byteorder="big") % n
    else:
        h = int.from_bytes(SHA3_256.new(m + r.to_bytes((r.bit_length() + 7) // 8, byteorder="big")).digest(), byteorder="big") % n
    s = (sL * h + k) % n

    return h, s

def ver(s, h, m, qA):
	global P
	sP = int(s) * P
	hA = int(h) * qA
	V = sP - hA
	v = V.x % n
	hnew = int.from_bytes(SHA3_256.new(m.encode("utf-8") + v.to_bytes((v.bit_length() + 7) // 8, byteorder= "big")).digest(), byteorder= "big") % n
	return h == hnew

def genKey():
    global n
    global P

    sA = random.randint(0, n - 1)
    qA = sA * P

    return (sA, qA)

stuID = 23631  ## 24198,19872, 23574, 25655

E = Curve.get_curve('secp256k1')
n = E.order
P = E.generator

#create a long term key
sL = 23927729476774547655045008832834504310921797729497501158914766546192068080855
ql = sL * P

lkey = ql

#

k = random.randint(1, n - 1)
h, s = sign(k, str(stuID).encode("utf-8"), False)

#Generate 10 Ephemeral Key
eKeys = []
for i in range(10):
    eKeys.append(genKey())

#server's long term key
QSer_long = Point(0xc1bc6c9063b6985fe4b93be9b8f9d9149c353ae83c34a434ac91c85f61ddd1e9 , 0x931bd623cf52ee6009ed3f50f6b4f92c564431306d284be7e97af8e443e69a8c, E)
QCli_long = lkey

###########DELETE LONG TERM KEY
# If you lost your long term key, you can reset it yourself with below code.
'''
# First you need to send a request to delete it. 
mes = {'ID': stuID}
response = requests.get('{}/{}'.format(API_URL, "RstLongRqst"), json = mes)
print(response)
code = input()

#Then server will send a verification code to your email. 
# Send this code to server using below code
mes = {'ID': stuID, 'CODE': code}
response = requests.get('{}/{}'.format(API_URL, "RstLong"), json = mes)
'''
#Now your long term key is deleted. You can register again. 
'''
####Register Long Term Key
mes = {'ID':stuID, 'H': h, 'S': s, 'LKEY.X': QCli_long.x, 'LKEY.Y': QCli_long.y}
response = requests.put('{}/{}'.format(API_URL, "RegLongRqst"), json = mes)
print(response.json())
code = input()

mes = {'ID':stuID, 'CODE': code}
response = requests.put('{}/{}'.format(API_URL, "RegLong"), json = mes)
print(response.json())
'''
###delete ephemeral keys
mes = {'ID': stuID, 'S': s, 'H': h}
response = requests.get('{}/{}'.format(API_URL, "RstEKey"), json = mes)

i = 0
for sA, qA in eKeys:
    #h, s = sign(randint(1, n - 2), qA.x.to_bytes((qA.x.bit_length() + 7) // 8, byteorder= "big") + qA.y.to_bytes((qA.y.bit_length() + 7) // 8, byteorder= "big"), False)
    h, s = sign(randint(1, n - 2), (str(qA.x) + str(qA.y)).encode("utf-8"), False)
    ekey = qA

    #send ephemeral key
    mes = {'ID': stuID, 'KEYID': i , 'QAI.X': ekey.x, 'QAI.Y': ekey.y, 'Si': s, 'Hi': h}
    response = requests.put('{}/{}'.format(API_URL, "SendKey"), json = mes)
    print(response.json())

    i += 1

k = random.randint(1, n - 1)
hSID, sSID = sign(k, str(stuID).encode("utf-8"), False)

for i in range(5):
    #Receiving Messages
    mes = {'ID_A': stuID, 'S': sSID, 'H': hSID}
    response = requests.get('{}/{}'.format(API_URL, "ReqMsg"), json = mes)
    print(response.json())
    response = response.json()

    ctext = response["MSG"].to_bytes((response["MSG"].bit_length() + 7) // 8, byteorder= "big")
    SEKey = Point(int(response["QBJ.X"]), int(response["QBJ.Y"]), E) #ephemeral public key of server
    #decrypt messages
    k = random.randint(1, n - 2)
    T = eKeys[int(response["KEYID"])][0] * SEKey
    U = str(T.x) + str(T.y) + "NoNeedToRunAndHide"
    KENC = SHA3_256.new(U.encode("utf-8"))
    KMAC = SHA3_256.new(KENC.digest())
    
    msg = ctext[:-32]
    h = HMAC.new(KMAC.digest(), digestmod=SHA256)
    h.update(msg[8:])
    
    try:
        h.verify(ctext[-32:])
    except :
        print("Message could not verified")
        continue

    aes = AES.new(KENC.digest(), AES.MODE_CTR, nonce= msg[0:8])
    dec = aes.decrypt(msg[8:])
    decMsg = str(dec, encoding= "utf-8")

    #send decrypted messages to server
    mes = {'ID_A': stuID, 'DECMSG': decMsg}
    response = requests.put('{}/{}'.format(API_URL, "Checker"), json = mes)
    print(response.text)